In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"lakshh21","key":"a2ae428b98ebadf82ed5c18961c17cd0"}'}

In [2]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 19% 5.00M/25.7M [00:00<00:00, 37.3MB/s]
100% 25.7M/25.7M [00:00<00:00, 124MB/s] 


In [3]:
import os
import json
import re

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [5]:
# Unzip dataset file
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zipObj:
  zipObj.extractall()

In [6]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [7]:
data = pd.read_csv('IMDB Dataset.csv')

In [8]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [11]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [12]:
data.isnull().sum()

,0
review,0
sentiment,0


In [13]:
# changing categorical data into numeric values
data.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)

<ipython-input-13-17222ac26716>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)


In [14]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [15]:
# splitting data into train and test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [16]:
train_data.shape

(40000, 2)

In [17]:
test_data.shape

(10000, 2)

In [18]:
# Data Preprocessing
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=600)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=600)

In [19]:
X_train

array([[   0,    0,    0, ...,  205,  351, 3856],
       [   0,    0,    0, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [20]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [   0,    0,    0, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [21]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [22]:
X_test.shape , X_train.shape , Y_test.shape , Y_train.shape

((10000, 600), (40000, 600), (10000,), (40000,))

In [23]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

In [24]:
# Building the model
model = Sequential()
model.add(Embedding(5000, 128, input_shape=(600,)))
model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.1)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 600, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 779,905 (2.98 MB)

 Trainable params: 779,905 (2.98 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=15,
          validation_split=0.2)

Epoch 1/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 591s 1s/step - accuracy: 0.6583 - loss: 3.2206 - val_accuracy: 0.8370 - val_loss: 0.4329
Epoch 2/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 616s 1s/step - accuracy: 0.8215 - loss: 0.4407 - val_accuracy: 0.7996 - val_loss: 0.4662
Epoch 3/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 627s 1s/step - accuracy: 0.8338 - loss: 0.4190 - val_accuracy: 0.8691 - val_loss: 0.3469
Epoch 4/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 621s 1s/step - accuracy: 0.8358 - loss: 0.4001 - val_accuracy: 0.8637 - val_loss: 0.3558
Epoch 5/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 620s 1s/step - accuracy: 0.8570 - loss: 0.3721 - val_accuracy: 0.8453 - val_loss: 0.3861
Epoch 6/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 622s 1s/step - accuracy: 0.8707 - loss: 0.3417 - val_accuracy: 0.8744 - val_loss: 0.3284
Epoch 7/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 625s 1s/step - accuracy: 0.8732 - loss: 0.3408 - val_accuracy: 0.8636 - val_loss: 0.3581
Epoch 8/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 619s 1s/step - accuracy: 0.8708 - loss: 0.3422 - val_accu

In [28]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 103s 328ms/step - accuracy: 0.8774 - loss: 0.3372
Test Loss: 0.3363, Test Accuracy: 0.8805


In [29]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review]) # taking all words in a sequence rather seperately
  padded_sequence = pad_sequences(sequence, maxlen=600)
  prediction = model.predict(padded_sequence)
  if prediction[0][0] > 0.5: # eg. [[0.45,0.55]] then this is a negative example
    sentiment = 'positive'
  else:
    sentiment = 'negative'
  return sentiment

In [30]:
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 664ms/step
The sentiment of the review is: positive


In [31]:
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
The sentiment of the review is: positive
